In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from time import sleep
import re
import requests
import csv
import os
from PIL import Image
from io import BytesIO
import ssl
import dload

In [2]:
!pip install requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/92/5e/50028bbb269986d9bc30270cd46b47ea44a1ca0b3f8da3a8429680d37050/charset_normalizer-3.3.0-cp311-cp311-win_amd64.whl.metadata
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/d2/b2/b157855192a68541a91ba7b2bbcb91f1b4faa51f8bae38d8005c034be524/urllib3-2.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-none-any.whl.metadata
  Using cached certifi-

In [4]:
import requests

url = "https://www.cmcsungmo.or.kr/api/attach/view/doctor/D0001029/thumbs/1"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers, verify=False)

if response.status_code == 200:
    image_data = response.content
    with open("test.jpg", "wb") as img_file:
        img_file.write(image_data)


SSLError: HTTPSConnectionPool(host='www.cmcsungmo.or.kr', port=443): Max retries exceeded with url: /api/attach/view/doctor/D0001029/thumbs/1 (Caused by SSLError(SSLError(1, '[SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1006)')))

In [2]:
department = "가정의학과"

# 1. chrome driver 띄우기
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 가톨릭대학교 서울성모병원 url
url = 'https://www.cmcseoul.or.kr/page/department/A'
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)
driver.get(url)

# 1. 과 검색

# 1.1. 검색 버튼 클릭
search_button = driver.find_element(By.CLASS_NAME, 'btn_search')
search_button.click()

# 1.2. 검색창 버튼 클릭
search_box = driver.find_element(By.ID, 'kwd')
time.sleep(3)
search_box.click()

# 1.3. 진료과 이름 입력
search_box.clear() 
search_box.send_keys(department)

# 1.4. 검색 버튼 클릭
search_button2 = driver.find_element(By.CLASS_NAME, 'search_btn')
search_button2.click()

# 1.5. 의료진 버튼 클릭
docker_link = driver.find_element(By.XPATH, '//a[contains(@href, "javascript:goCategory(\'DOCTOR\');")]')
docker_link.click() 

# 2. 의료진 정보 가져오기

# 2.1. html 파싱
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soup

driver.quit()

<html lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>가톨릭대학교 서울성모병원</title>
<link href="/ui_common/css/reset.css" rel="stylesheet" type="text/css"/>
<link href="/ui_common/css/ui.common.css" rel="stylesheet" type="text/css"/>
<link href="/ui_common/css/ui.common.seoul.css" rel="stylesheet" type="text/css"/>
<link href="/ui_common/css/jquery.mCustomScrollbar.css" rel="stylesheet" type="text/css"/>
<link href="/ui_common/css/konan.sf.css" rel="stylesheet" type="text/css"/>
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-1GJN3LC9EW&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script src="/ui_common/js/jquery-1.12.4.min.js" type="text/javascript"></script>
<script src="/ui_common/js/jquery.easing.1.3.js" type="text/javascript"></script>
<script src="/ui_common/js/TweenMax.js" type="text/javascript"></sc

In [13]:

doctor_tags = soup.find_all('div', class_='doc_list_wrap')
dptm_name_elem = doctor_tags[1].find('strong').text
name_elem, dptm_elem = dptm_name_elem.split('[')

name = name_elem.strip()
dptm = dptm_elem.strip('[]')

img_elem = doctor_tags[1].find('img')['src']
img_name = "가톨릭대학교" + " " + "서울성모병원" + " " + dptm + " " + name + ".png"
img_src = "https://www.cmcseoul.or.kr" + img_elem

print(img_name)
print(img_src)

# 이미지 다운로드 ver. 1
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
}

response = requests.get(img_src, headers=headers, verify=False)
if response.status_code == 200:
    image_data = response.content
    image = Image.open(BytesIO(image_data))
    image.save(f"03/images/{img_name}") 

# 이미지 다운로드 ver. 2
context = ssl._create_unverified_context()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
}

try:
    with urllib.request.urlopen(img_src, context=context) as response, open(f"03/images/{img_name}", 'wb') as out_file:
        data = response.read()
        out_file.write(data)
except urllib.error.URLError as e:
    print(f"Failed to download the image: {e}")
    
    context = ssl._create_unverified_context()

    
    


가톨릭대학교 서울성모병원 가정의학과 최환석.png
https://www.cmcseoul.or.kr/api/attach/view/doctor/D0000872/thumbs/1


SSLError: HTTPSConnectionPool(host='www.cmcseoul.or.kr', port=443): Max retries exceeded with url: /api/attach/view/doctor/D0000872/thumbs/1 (Caused by SSLError(SSLError(1, '[SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1007)')))

In [3]:
url = "https://www.cmcsungmo.or.kr/api/attach/view/doctor/D0001029/thumbs/1"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
}

# Create a request with the provided headers
req = urllib.request.Request(url, headers=headers)

# Time check
start = time.time()

# Download the image
response = urllib.request.urlopen(req)
with open("test.jpg", "wb") as img_file:
    img_file.write(response.read())

# Check the time taken for image download
print(time.time() - start)

# Open the saved image
img = Image.open("test.jpg")


URLError: <urlopen error [SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1007)>

In [2]:
url = "https://www.cmcsungmo.or.kr/api/attach/view/doctor/D0001029/thumbs/1"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers, verify=False)

if response.status_code == 200:
    image_data = response.content
    with open("test.jpg", "wb") as img_file:
        img_file.write(image_data)

img = Image.open("test.jpg")

SSLError: HTTPSConnectionPool(host='www.cmcsungmo.or.kr', port=443): Max retries exceeded with url: /api/attach/view/doctor/D0001029/thumbs/1 (Caused by SSLError(SSLError(1, '[SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1007)')))

In [ ]:
import dload

# 이미지 다운로드 및 저장할 경로 지정
url = "https://www.cmcsungmo.or.kr/api/attach/view/doctor/D0001029/thumbs/1"
save_path = "test2.jpg"
__file__ = "03_img_test.ipynb"

# 이미지 다운로드
dload.save(url, save_path)


None


Traceback (most recent call last):
  File "c:\Users\hxhki\anaconda3\envs\bigdata_env\lib\site-packages\urllib3\connectionpool.py", line 467, in _make_request
    self._validate_conn(conn)
  File "c:\Users\hxhki\anaconda3\envs\bigdata_env\lib\site-packages\urllib3\connectionpool.py", line 1092, in _validate_conn
    conn.connect()
  File "c:\Users\hxhki\anaconda3\envs\bigdata_env\lib\site-packages\urllib3\connection.py", line 642, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "c:\Users\hxhki\anaconda3\envs\bigdata_env\lib\site-packages\urllib3\connection.py", line 783, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
  File "c:\Users\hxhki\anaconda3\envs\bigdata_env\lib\site-packages\urllib3\util\ssl_.py", line 469, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(sock, context, tls_in_tls, server_hostname)
  File "c:\Users\hxhki\anaconda3\envs\bigdata_env\lib\site-packages\urllib3\util\ssl_.py", line 513, in _ssl_wrap_socket

''

In [3]:
# 1. chrome driver 띄우기
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 가톨릭대학교 서울성모병원 url
url = 'https://www.cmcseoul.or.kr/page/department/A'
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)
driver.get(url)

mouse = driver.find_elements(By.CLASS_NAME, 'front')
button = driver.find_elements(By.CLASS_NAME, 'btn-bak1')

act = ActionChains(driver)
act.move_to_element(mouse[0]).click(button[0]).perform()

In [4]:
doctor_tags = driver.find_elements(By.CSS_SELECTOR, 'div.doc_list_wrap')
doctor_tags

[<selenium.webdriver.remote.webelement.WebElement (session="fcc4de8df6fec527d095cc6fd40e09a5", element="E08CED0111E1B1D6B6BDC9489A7BC716_element_158")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fcc4de8df6fec527d095cc6fd40e09a5", element="E08CED0111E1B1D6B6BDC9489A7BC716_element_159")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fcc4de8df6fec527d095cc6fd40e09a5", element="E08CED0111E1B1D6B6BDC9489A7BC716_element_160")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fcc4de8df6fec527d095cc6fd40e09a5", element="E08CED0111E1B1D6B6BDC9489A7BC716_element_161")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fcc4de8df6fec527d095cc6fd40e09a5", element="E08CED0111E1B1D6B6BDC9489A7BC716_element_162")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fcc4de8df6fec527d095cc6fd40e09a5", element="E08CED0111E1B1D6B6BDC9489A7BC716_element_163")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fcc4de8df6fec527d095cc

In [5]:
image = doctor_tags[0].find_element(By.CSS_SELECTOR, 'span.resize_img-b')
image

<selenium.webdriver.remote.webelement.WebElement (session="fcc4de8df6fec527d095cc6fd40e09a5", element="E08CED0111E1B1D6B6BDC9489A7BC716_element_166")>

In [8]:
act = ActionChains(driver)
act.move_to_element(image).context_click().perform()

In [10]:
act.send_keys('v').perform()